In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt 
import numpy as np 
import read_filepath_module

In [ ]:
# 确定两种数据源的路径
# 微波辐射计数据时段包含探空气球数据的时段,所以由探空气球代表的的世界时追踪微波辐射计代表的北京时
Year = '2010'
Filepath = r'I:\Data\Personal Data\graduation project\SACOL'
# ↓探空气球数据
filepath_1 = Filepath + '\Balloon'
Target_str_1 = 'UPAR'+Year
# ↓微波辐射计数据
filepath_2 = Filepath + '\microwave'
Target_str_2 = 'lv2_'+Year


In [ ]:
'''
    北京时 = 世界时 + 8
    此函数的功能就是将世界时转换为对应的北京时，
'''
def f_trans(str):
    if(str == '0000'):
        return('0800')
    elif(str == '1200'):
        return('2000')
    else:
        return('Error')

In [ ]:
def search_data(filepath):
    # data_1是气球探空数据
    data_1 = pd.read_csv(filepath)
    # data_2 暂时存储同一时间的匹配数据
    data_2 = pd.DataFrame()
    output_data = pd.DataFrame()
    i = 0
    while i < data_1.shape[0]:
    # while i < 1:
        # 清空绘图数据
        data_2.drop(data_2.index, inplace=True)
        data_2 = data_2.append(data_1.loc[i:i,])
        file_search_point = 'lv2_'+str(data_1.loc[i:i,'Time'].values)[2:8]
        file_search_path = filepath_2 + '\\' + str(data_1.loc[i:i,'Time'].values)[2:6] +'\\' + file_search_point + '.csv'
        if(os.path.exists(file_search_path)):
            # data_3 载入搜索文件
            # 搜索并标记
            data_3 = pd.read_csv(file_search_path)
            data_3.rename(columns={'Date/Time':'Time'},inplace=True)
            # 转换搜索点
            trans_point = str(data_1.loc[i:i,'Time'].values)[2:10] + '_WT' + f_trans(str(data_1.loc[i:i,'Time'].values)[14:18])
            j = -1
            flag = -1
            # 遍历搜索文件寻找搜索点
            while j < data_3.shape[0]-1:
                j = j + 1
                T = str(data_3.loc[j:j,'Time'].values)[2:17]
                if(T == trans_point):
                    flag = j
                    data_2 = data_2.append(data_3.loc[j:j,])
                # else:
                #     print(trans_point[0:8]+'_BjT'+trans_point[11:18] +' NO exists!')
                #     print(T+'\n')
        # -----------------------------------------------------------
            if(flag == -1):
                print('No compare data:'+trans_point)
                # print('No compare data: '+trans_point[0:8]+'_BjT'+trans_point[11:18])
                break
            else:
                output_data = output_data.append(data_2)
                print('Ready to find: '+trans_point)
                # print('Ready to find: '+trans_point[0:8]+'_BjT'+trans_point[11:18])
        else:
            print('No exits '+ file_search_path+'\n')
            break
        i = i + 1
    output_data = output_data.round(2)
    return(output_data)

In [ ]:
# 气球探空数据
L1_filepaths = read_filepath_module.read_filepath(filepath_1,Target_str_1)
# 微波辐射计探空数据
L2_filepaths = read_filepath_module.read_filepath(filepath_2,Target_str_2)
data = pd.DataFrame()
for L1_filepath in L1_filepaths:
    data = data.append(search_data(L1_filepath))
    print('Finish:'+L1_filepath)
data.reset_index(inplace=True,drop=True)
# data.to_csv(r'H:\桌面\test.csv')

In [ ]:
# col = 1
# while col<data.shape[1]:
#     row = 0
#     while row<data.shape[0]:
#         data.iloc[row:row+1,col:col+1] = (data.iloc[row:row+1,col:col+1]>350)
#         row=row+1
#     col=col+1
# row = 502
# col = 4
# print(False in (data.iloc[row:row+1,col:col+1]>350).values)

In [ ]:
def delete_row(row):
    # flag = flag + 1
    if(row&1 == 0):
        data.drop([row,row+1],inplace=True)
        print(f'Have already delete {row} and {row+1}')
    else:
        data.drop([row-1,row],inplace=True)
        print(f'Have already delete {row-1} and {row}')
#---------------------------------------------------------
flag = 0
col = 1
while col<data.shape[1]:
    #print('col'+str(col))
    row = 0
    data.reset_index(inplace=True,drop=True)
    while row<data.shape[0]:
        # print('    row'+str(row))
        # print(data.iloc[row:row+1,col:col+1])
        if(str(data.iloc[row:row+1,col:col+1].values)[2:5] == 'nan'):
            print('NaN')
            print(row,col)
            delete_row(row)
            if(row&1 ==0):
                row = row + 2
            else:
                row = row + 1
        elif(True in (data.iloc[row:row+1,col:col+1]>350).values):
            print(row,col)
            delete_row(row)
            if(row&1 ==0):
                row = row + 2
            else:
                row = row + 1
        else:
            row = row + 1
    col = col + 1

In [ ]:
def MSE(df):
    npdata =np.array(df)
    # print(npdata)
    row = 0
    sum = 0
    while row<npdata.shape[0]:
        sum = sum + (npdata[row:row+1,:]-npdata[row+1:row+2,:]) ** 2
        # print(npdata[row:row+1,:])
        # print(npdata[row+1:row+2,:])
        row = row + 2
    MSE_data = sum/npdata.shape[0]
    return(MSE_data)

In [ ]:
col = 1
data1123n = [ ]
while col < data.shape[1]:
    # print(MSE(data.iloc[:,col:col+1]).round(2))
    print(float(MSE(data.iloc[:,col:col+1]).round(2)))
    data1123n.append(float(MSE(data.iloc[:,col:col+1]).round(2)))
    col = col + 1


In [ ]:
photoname = Year
plt.rcParams['figure.figsize'] = (10.0,10.0)
#纵轴上分布高度数值
Height = ['0.00', '0.10', '0.20', '0.30', '0.40', '0.50', '0.60', '0.70', '0.80', '0.90', '1.00', '1.25', '1.50', '1.75', '2.00', '2.25', '2.50', '2.75', '3.00', '3.25', '3.50', '3.75', '4.00', '4.25', '4.50', '4.75', '5.00', '5.25', '5.50', '5.75', '6.00', '6.25', '6.50', '6.75', '7.00', '7.25', '7.50', '7.75', '8.00', '8.25', '8.50', '8.75', '9.00', '9.25', '9.50', '9.75', '10.00']
# 绘图
plt.plot(data1123n, Height)
#设置X轴
x_ticks = np.linspace(0,120,13)
plt.xticks(x_ticks,rotation=45)
plt.xlabel("MSE")
plt.ylabel("Altitude km")
# 启动图例
# plt.legend()
# 添加网格
plt.grid()
# 主标题
plt.title(photoname,fontsize=20,fontweight='bold')
# 保存图片
photopath = 'G:/python/Graduation design/photo/' + photoname + '.png'
# plt.savefig(photopath,bbox_inches = 'tight')
# 展示图片
plt.show()
